In [1]:
# Step 1: Install dependencies
!pip install gradio transformers requests beautifulsoup4 textblob

# Step 2: Imports
import gradio as gr
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
import re
from textblob import TextBlob
import numpy as np

# Step 3: Load sentiment pipeline
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)

# Step 4: Improved text extraction with comment collection
def extract_comments_from_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        comments = []

        # Try to find comment sections (common patterns)
        comment_selectors = [
            '[class*="comment"]',
            '[class*="review"]',
            '[class*="user-content"]',
            '[class*="user-text"]',
            '.comment-text',
            '.review-text',
            '.user-comment',
            '.comment-content',
            '.review-content',
            '.comment-body'
        ]

        for selector in comment_selectors:
            elements = soup.select(selector)
            for element in elements:
                comment_text = element.get_text(strip=True)
                if len(comment_text) > 20:  # Only take substantial comments
                    comments.append(comment_text)

        # If no specific comments found, try to extract paragraphs as individual comments
        if len(comments) < 3:
            paragraphs = soup.find_all('p')
            for p in paragraphs:
                p_text = p.get_text(strip=True)
                if len(p_text) > 30 and len(p_text) < 500:  # Reasonable comment length
                    comments.append(p_text)

        # Remove duplicates and clean
        comments = list(set(comments))
        comments = [c for c in comments if len(c) > 10]

        return comments[:20]  # Limit to 20 comments max

    except Exception as e:
        return [f"Error: Could not extract comments - {str(e)}"]

# Step 5: Enhanced sentiment analysis for multiple comments
def analyze_comments(input_text_or_url):
    # Determine if input is URL or text
    if input_text_or_url.startswith("http"):
        if "imdb.com" in input_text_or_url and "/video/" in input_text_or_url:
            return "❌ Video pages don't have comments. Try IMDb review pages instead!"

        comments = extract_comments_from_url(input_text_or_url)
        if not comments or "Error" in comments[0]:
            return f"❌ Could not extract comments from this page. Please try a different URL or paste comments directly."

        source_info = f"📊 Analyzed {len(comments)} comments from the webpage"

    else:
        # Split text input by lines or periods to get multiple comments
        text = input_text_or_url
        # Split by newlines first, then by sentences if needed
        if '\n' in text:
            comments = [line.strip() for line in text.split('\n') if len(line.strip()) > 10]
        else:
            # Split by sentences
            comments = [s.strip() for s in text.split('.') if len(s.strip()) > 10]

        if len(comments) < 2:
            return "❌ Please provide multiple comments (separated by newlines) or a webpage with comments for comparison."

        source_info = f"📊 Analyzed {len(comments)} comments from your input"

    if len(comments) < 2:
        return "❌ Not enough comments found for comparison. Please provide more content."

    # Analyze each comment
    results = []
    sentiment_counts = {"positive": 0, "negative": 0, "neutral": 0}
    confidence_scores = []

    for i, comment in enumerate(comments[:15]):  # Limit to 15 comments for performance
        try:
            result = sentiment_pipeline(comment[:400])[0]  # Shorter limit for multiple comments
            label = result['label']
            score = result['score']

            # Classify sentiment
            if "1 star" in label or "2 stars" in label:
                sentiment = "negative"
                emoji = "👎"
            elif "3 stars" in label:
                sentiment = "neutral"
                emoji = "😐"
            elif "4 stars" in label or "5 stars" in label:
                sentiment = "positive"
                emoji = "👍"
            else:
                sentiment = "neutral"
                emoji = "🤔"

            sentiment_counts[sentiment] += 1
            confidence_scores.append(score)

            results.append({
                "comment": comment[:100] + "..." if len(comment) > 100 else comment,
                "sentiment": sentiment,
                "emoji": emoji,
                "confidence": score
            })

        except Exception as e:
            continue  # Skip comments that fail analysis

    if not results:
        return "❌ Could not analyze any comments. Please try different content."

    # Calculate overall statistics
    total = len(results)
    positive_pct = (sentiment_counts["positive"] / total) * 100
    negative_pct = (sentiment_counts["negative"] / total) * 100
    neutral_pct = (sentiment_counts["neutral"] / total) * 100
    avg_confidence = np.mean(confidence_scores) * 100

    # Determine overall reaction
    if positive_pct > 60:
        overall = "OVERWHELMINGLY POSITIVE 🎉"
        reaction_emoji = "😍"
        summary = "People loved this content! The majority had very positive things to say."
    elif positive_pct > 40:
        overall = "MOSTLY POSITIVE 😊"
        reaction_emoji = "🙂"
        summary = "The general reaction was positive. More people liked it than didn't."
    elif negative_pct > 60:
        overall = "OVERWHELMINGLY NEGATIVE 😞"
        reaction_emoji = "😠"
        summary = "People were not happy with this content. The majority had negative feedback."
    elif negative_pct > 40:
        overall = "MOSTLY NEGATIVE 👎"
        reaction_emoji = "😐"
        summary = "The general reaction was negative. More people had issues than praised it."
    elif neutral_pct > 50:
        overall = "MIXED/NEUTRAL 🤔"
        reaction_emoji = "🤷"
        summary = "People had mixed feelings. No strong consensus either way."
    else:
        overall = "DIVIDED OPINIONS ⚖️"
        reaction_emoji = "⚖️"
        summary = "The audience is split between positive and negative reactions."

    # Build detailed report
    report = f"""
{reaction_emoji} **OVERALL REACTION: {overall}**

{summary}

{source_info}
---

📈 **SENTIMENT BREAKDOWN:**
✅ Positive: {sentiment_counts['positive']} comments ({positive_pct:.1f}%)
❌ Negative: {sentiment_counts['negative']} comments ({negative_pct:.1f}%)
⚖️ Neutral: {sentiment_counts['neutral']} comments ({neutral_pct:.1f}%)

🎯 Analysis Confidence: {avg_confidence:.1f}%

---

💬 **SAMPLE COMMENTS:**

"""

    # Add sample comments from each category
    for sentiment_type in ["positive", "negative", "neutral"]:
        type_comments = [r for r in results if r["sentiment"] == sentiment_type]
        if type_comments:
            if sentiment_type == "positive":
                report += f"**👍 Positive Comments:**\n"
            elif sentiment_type == "negative":
                report += f"**👎 Negative Comments:**\n"
            else:
                report += f"**😐 Neutral Comments:**\n"

            for i, result in enumerate(type_comments[:2]):  # Show 2 examples per type
                report += f"{i+1}. {result['emoji']} \"{result['comment']}\"\n"
            report += "\n"

    report += f"\---\n**💡 Insight:** "

    if positive_pct > 70:
        report += "This content really resonated with the audience!"
    elif negative_pct > 70:
        report += "There are significant issues that need addressing."
    elif neutral_pct > 50:
        report += "The content didn't strongly impress or disappoint most people."
    else:
        report += "Opinions are varied - different people had different experiences."

    return report

# Step 6: Build the enhanced UI
iface = gr.Interface(
    fn=analyze_comments,
    inputs=gr.Textbox(
        lines=10,
        placeholder="Paste multiple comments (one per line) OR a webpage URL with comments...\n\nExamples:\n• 'Amazing movie! Best of the year!\\nTerrible acting, would not recommend.\\nIt was okay, nothing special.'\n• https://www.imdb.com/title/tt0111161/reviews/\n• Any news article or blog post with comments",
        label="Enter Multiple Comments or Website URL"
    ),
    outputs=gr.Textbox(
        lines=15,
        label="Audience Reaction Analysis",
        show_copy_button=True
    ),
    title="🎬 Multi-Comment Sentiment Analyzer",
    description="Analyze multiple comments together to understand overall audience reaction! Paste comments (one per line) or a webpage URL with comments.",
    examples=[
        ["Loved this movie! Great acting.\nThe plot was boring and predictable.\nAmazing visuals but weak story.\nBest film I've seen this year!\nNot worth the money, very disappointed."],
        ["https://www.imdb.com/title/tt0111161/reviews/"],
        ["Outstanding performance by all actors.\nThe cinematography was breathtaking.\nSome scenes felt too long and dragged.\nExcellent direction and screenplay.\nAverage movie, expected more."]
    ],
    theme="soft"
)

# Step 7: Launch UI
iface.launch(share=True)

<>:210: SyntaxWarning: invalid escape sequence '\-'
<>:210: SyntaxWarning: invalid escape sequence '\-'
/tmp/ipython-input-3135310056.py:210: SyntaxWarning: invalid escape sequence '\-'
  report += f"\---\n**💡 Insight:** "


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://63b0dd944c05dacd13.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
